In [21]:
macro_data_path="/data3/hcxia/Adahist2/datasets/macro"
# read files from macro_data_path
import os
import pandas as pd
import numpy as np
# some files are csv and some are txt 
# read all files in the directory
def read_files_from_path(path):
    files = os.listdir(path)
    data = {}
    for file in files:
        print(file)
        if file.endswith(".csv"):
            data[file] = pd.read_csv(os.path.join(path, file))
        elif file.endswith(".txt"):
            data[file] = pd.read_table(os.path.join(path, file))
    return data
# print the first 5 rows of each file
data = read_files_from_path(macro_data_path)
for file in data:
    print(file)
    print(data[file].head(1))
    print("\n\n")

DKW_updates.csv
lmci_feds.csv
macro1_Monthly.txt
ebp_csv.csv
FED_Note_Term_SOFR.csv
fci_g_public_monthly_3yr.csv
macro1_Quarterly.txt
FEDS-Note-2873-cie-data.csv
DKW_updates.csv
         date  exp.real.short.rate.5  exp.inflation.5  real.term.prem.5  \
0  1983-01-03                  3.761           4.2453            1.8474   

   inflation.risk.prem.5  tips.liq.prem.5  nominal.yield.raw.5  \
0                 0.3225              NaN              10.2174   

   nominal.yield.fitted.5  ic.raw.5  ic.fitted.5  ...  ic.fitted.10  \
0                 10.1762       NaN          NaN  ...           NaN   

   exp.real.short.rate.5f5  exp.inflation.5f5  real.term.prem.5f5  \
0                   3.2994             3.7673               2.904   

   inflation.risk.prem.5f5  tips.liq.prem.5f5  nominal.yield.raw.5f5  \
0                 0.741499                NaN                10.6264   

   nominal.yield.fitted.5f5  ic.raw.5f5  ic.fitted.5f5  
0                 10.712199         NaN            NaN

In [22]:
# unify the column names
# change "date","DATE","period" to "date"

for file in data:
    if "date" in data[file].columns:
        data[file] = data[file].rename(columns={"date":"date"})
    elif "DATE" in data[file].columns:
        data[file] = data[file].rename(columns={"DATE":"date"})
    elif "period" in data[file].columns:
        data[file] = data[file].rename(columns={"period":"date"})
    else:
        print("no date column in {}".format(file))

import pandas as pd

# List of possible date formats
date_formats = [
    "%Y-%m-%d",    # 1999-3-31, 2000-01-01
    "%d-%m-%Y",    # 05-12-2021
    "%Ym%m",       # 1976m8
]

# Iterate through the files
for file in data:
    if "date" in data[file].columns:
        date_column = data[file]["date"]
        converted_date = None
        print(f"Converting dates in {file}")
        print("date before conversion",date_column.head(5))

        # Try each format
        for fmt in date_formats:
            try:
                converted_date = pd.to_datetime(date_column, format=fmt, errors='coerce')
                # Break if parsing succeeds (i.e., no more NaT values)
                if not converted_date.isna().all():
                    break
            except Exception as e:
                continue

        # Fallback to automatic parsing for unhandled formats
        if converted_date is None or converted_date.isna().any():
            try:
                converted_date = pd.to_datetime(date_column, errors='coerce')
            except Exception as e:
                print(f"Error parsing dates in file {file}: {e}")

        # Check for remaining NaT values and log them
        if converted_date.isna().any():
            print(f"Date format error in {file}: ", data[file].loc[converted_date.isna(), "date"].values)

        # Retain only month and year
        data[file]["date"] = converted_date.dt.to_period('M').dt.to_timestamp()
        print(f"Converted dates in {file} to {converted_date.dt.to_period('M').dt.to_timestamp().dt.strftime('%Y-%m')}")

    else:
        print(f"No date column in {file}")

    

Converting dates in DKW_updates.csv
date before conversion 0    1983-01-03
1    1983-01-04
2    1983-01-05
3    1983-01-06
4    1983-01-07
Name: date, dtype: object
Converted dates in DKW_updates.csv to 0        1983-01
1        1983-01
2        1983-01
3        1983-01
4        1983-01
          ...   
10909    2024-10
10910    2024-10
10911    2024-10
10912    2024-10
10913    2024-10
Name: date, Length: 10914, dtype: object
Converting dates in lmci_feds.csv
date before conversion 0     1976m8
1     1976m9
2    1976m10
3    1976m11
4    1976m12
Name: date, dtype: object
Converted dates in lmci_feds.csv to 0      1976-08
1      1976-09
2      1976-10
3      1976-11
4      1976-12
        ...   
486    2017-02
487    2017-03
488    2017-04
489    2017-05
490    2017-06
Name: date, Length: 491, dtype: object
Converting dates in macro1_Monthly.txt
date before conversion 0    2000-01-01
1    2000-02-01
2    2000-03-01
3    2000-04-01
4    2000-05-01
Name: date, dtype: object
Converted dat

In [23]:
# cast all non-date columns to numeric
for file in data:
    for column in data[file].columns:
        if column != "date":
            data[file][column] = pd.to_numeric(data[file][column], errors='coerce')

In [24]:
# group mean by date
for file in data:
    data[file] = data[file].groupby("date").mean()
    # print(data[file].head(5))
    # still use date as a column
    data[file].reset_index(inplace=True)

In [25]:
# merge all data by date column
merged_data = None
for file in data:
    if merged_data is None:
        merged_data = data[file]
    else:
        merged_data = pd.merge(merged_data, data[file], on="date", how="outer")

In [26]:
# select date range from 2000-01 to 2024-12
merged_data = merged_data[(merged_data["date"] >= "2000-01") & (merged_data["date"] <= "2024-12")]

In [27]:
# describe the data,check nan values of each column
# foward fill nan values
merged_data = merged_data.fillna(method='ffill')
print(merged_data.describe())

                                date  exp.real.short.rate.5  exp.inflation.5  \
count                            298             298.000000       298.000000   
mean   2012-05-16 15:37:26.979865856               0.145329         2.401687   
min              2000-01-01 00:00:00              -0.901237         1.670877   
25%              2006-03-08 18:00:00              -0.686161         2.085608   
50%              2012-05-16 12:00:00              -0.117818         2.424196   
75%              2018-07-24 06:00:00               0.854036         2.676981   
max              2024-10-01 00:00:00               2.443314         3.298245   
std                              NaN               0.947135         0.370176   

       real.term.prem.5  inflation.risk.prem.5  tips.liq.prem.5  \
count        298.000000             298.000000       298.000000   
mean           0.210198              -0.024039         0.499732   
min           -0.376059              -0.252436        -1.186130   
25%        

/tmp/ipykernel_161489/2315347145.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_data = merged_data.fillna(method='ffill')


In [28]:
# save the merged data to a csv file
data_folder = "/data3/hcxia/Adahist2/datasets/macro_processed"
# change the 'date' column to "Date"
merged_data = merged_data.rename(columns={"date":"Date"})
# create the folder if it does not exist
if not os.path.exists(data_folder):
    os.makedirs(data_folder)
merged_data.to_csv(os.path.join(data_folder, "macro_data.csv"), index=False)

In [29]:
#read all stock data from the folder
import os
import pandas as pd
stock_data_path = "/data3/hcxia/Adahist2/datasets/yahoofinance_day_prices_future_etfs"
# read all csv files in the directory
def read_files_from_path(path):
    files = os.listdir(path)
    data = {}
    # print(files)
    for file in files:
        if file.endswith(".csv"):
            file_name = file.split(".")[0]
            data[file_name] = pd.read_csv(os.path.join(path, file))
    return data
# print the first 5 rows of each file
data = read_files_from_path(stock_data_path)
# concatenate all stock data together into one dataframe, and add a column "ticker" to indicate the stock
stock_data = None
for file in data:
    ticker = file.split(".")[0]
    if stock_data is None:
        stock_data = data[file]
        stock_data["ticker"] = ticker
    else:
        # merge the stock data based on the date column, with the file_name as the column name prefix
        # stock_data = pd.merge(stock_data, data[file], on="Date", how="outer", suffixes=("", "_"+ticker))
        # concatenate the stock data raw by raw
        stock_data_ = data[file]
        stock_data_["ticker"] = ticker
        stock_data = pd.concat([stock_data, data[file]], axis=0)
unique_dates=stock_data["Date"].unique()
print("unique dates",len(unique_dates))
# drop the row where Date is NaN
stock_data = stock_data.dropna(subset=["Date"])
print(stock_data.columns)
print(stock_data.head(5))
print("date range of stock data",stock_data["Date"].min(),stock_data["Date"].max())
print("size of stock data",stock_data.shape)
print("unique tickers",stock_data["ticker"].unique())
# print the NaN numb of each column
print(stock_data.isna().sum())
# save the merge stock data to a csv file
# sort the stock data by date
stock_data = stock_data.sort_values("Date")
data_folder = "/data3/hcxia/Adahist2/datasets/stock_data"
# create the folder if it does not exist
if not os.path.exists(data_folder):
    os.makedirs(data_folder)
stock_data.to_csv(os.path.join(data_folder, "megred_stock_data.csv"), index=False)

unique dates 6038
Index(['Date', 'Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume',
       'ticker'],
      dtype='object')
                        Date           Adj Close               Close  \
1  2005-12-12 00:00:00+00:00  104.90389251708984  119.55999755859375   
2  2005-12-13 00:00:00+00:00   104.8951644897461  119.55000305175781   
3  2005-12-14 00:00:00+00:00  105.37773895263672   120.0999984741211   
4  2005-12-15 00:00:00+00:00  105.07941436767578  119.76000213623047   
5  2005-12-16 00:00:00+00:00  105.48299407958984  120.22000122070312   

                 High                 Low                Open  Volume ticker  
1   119.8499984741211   119.4800033569336  119.62000274658203  601000    FXE  
2   119.9000015258789  119.30000305175781  119.38999938964844  155100    FXE  
3  120.58000183105469  120.02999877929688  120.43000030517578  441400    FXE  
4  120.06999969482422  119.44999694824219  119.98999786376953  562400    FXE  
5   120.2300033569336   119.9800033569336   

In [30]:
import pandas as pd
# merge the stock data with the macro data
stock_data_path="/data3/hcxia/Adahist2/datasets/stock_data/megred_stock_data.csv"
selected_macro_path = "/data3/hcxia/Adahist2/generator/GRT_GAN/data/macro_list.txt"
with open(selected_macro_path, "r") as f:
    Macro_features = f.read().splitlines()
macro_data_path="/data3/hcxia/Adahist2/datasets/macro_processed/macro_data.csv"
# merge the stock data with the macro data
stock_data = pd.read_csv(stock_data_path)
full_macro_data = pd.read_csv(macro_data_path)
selected_macro_data = full_macro_data[['Date']+Macro_features]
print("date stamps of macro data",selected_macro_data.head(10))
# now selected_macro_data only have the first day of each month, we need to forward fill the data to all days in the month
selected_macro_data["Date"] = pd.to_datetime(selected_macro_data["Date"])
selected_macro_data.set_index("Date", inplace=True)
selected_macro_data = selected_macro_data.resample("D").ffill()
print("resampled macro data",selected_macro_data.head(10))
# save the resampled macro data to a csv file
data_folder = "/data3/hcxia/Adahist2/datasets/macro_processed"
# create the folder if it does not exist
if not os.path.exists(data_folder):
    os.makedirs(data_folder)
selected_macro_data.to_csv(os.path.join(data_folder, "macro_data_resampled.csv"))
# merge the stock data with the macro data by date
# stock_data["Date"] = pd.to_datetime(stock_data["Date"])
# selected_macro_data.index = selected_macro_data.index.tz_localize(None)
# stock_data["Date"] = stock_data["Date"].dt.tz_localize(None)
# stock_data = pd.merge(stock_data, selected_macro_data, left_on="Date", right_index=True, how="left")
# #print size of stock data
# print("size of stock data",stock_data.shape)
# print("timestime range of stock data",stock_data["Date"].min(),stock_data["Date"].max())
# # save the merged stock data to a csv file
# stock_data_path = "/data3/hcxia/Adahist2/datasets/stock_data/stock_data_with_macro.csv"
# stock_data.to_csv(stock_data_path, index=False)


date stamps of macro data          Date  exp.real.short.rate.5  exp.inflation.5  real.term.prem.5  \
0  2000-01-01               2.199755         3.298245          0.932425   
1  2000-02-01               2.314455         3.255515          0.914670   
2  2000-03-01               2.315948         3.216604          0.810165   
3  2000-04-01               2.254247         3.162505          0.718184   
4  2000-05-01               2.443314         3.259314          0.829359   
5  2000-06-01               2.323250         3.183286          0.714214   
6  2000-07-01               2.304120         3.169460          0.653780   
7  2000-08-01               2.294400         3.134713          0.564448   
8  2000-09-01               2.219770         3.145145          0.533695   
9  2000-10-01               2.196948         3.132933          0.450248   

   inflation.risk.prem.5  tips.liq.prem.5  nominal.yield.raw.5  \
0               0.113740         1.260895             6.524823   
1               

/tmp/ipykernel_161489/3414964996.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_macro_data["Date"] = pd.to_datetime(selected_macro_data["Date"])
